## Time Series Streamlit Application


This application shows the actual versus predicted for any selected series. It also show the aggregated metrics.
The app is designed to work in Streamlit for Snowflake. 

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import altair as alt
from snowflake.snowpark.functions import col
from snowflake.snowpark.context import get_active_session

# Write directly to the app
st.title("Multi-Series Evaluation App")
st.write(
    """This app provides overall metrics as well as per series evaluation
    """
)

# Get the current credentials
session = get_active_session()

#Get Data
train_sdf = session.read.table('TS_M5_train')
train = train_sdf.to_pandas()
train['DS'] = pd.to_datetime(train['DS'])
train['DS'] = train['DS'].dt.strftime('%Y-%m-%d')
#st.write(train.head()) 

forecast_sdf = session.read.table('TS_M5_forecast_ML_UDTF')
forecast_sdf1 = session.read.table('TS_M5_forecast_Stats')
total = forecast_sdf.union(forecast_sdf1)
forecast = total.to_pandas()
forecast['DS'] = pd.to_datetime(forecast['DS'])
forecast['DS'] = forecast['DS'].dt.strftime('%Y-%m-%d')


#Series Selector
unique_ids = train['UNIQUE_ID'].unique()
selected_id = st.selectbox("Select a Unique ID", unique_ids)

#Algorithm Selector
unique_ag = forecast['ALGORITHM'].unique()
selected_ag = st.selectbox("Select a Algorithm", unique_ag)

##Get Metrics 
metricsbyseries_sdf = session.read.table('TS_M5_metrics_series')
metricsbyseriesdf = metricsbyseries_sdf.to_pandas()


#Filter Data
actuals = train[train['UNIQUE_ID'] == selected_id]
#st.dataframe(actuals)

filtered_forecast = forecast[forecast['UNIQUE_ID'] == selected_id]
predicted = filtered_forecast[['DS','VALUE']]
#st.dataframe(predicted)

metricsbyseries = metricsbyseriesdf[metricsbyseriesdf['UNIQUE_ID'] == selected_id]
metricsbyseries = metricsbyseries.pivot_table(index=['UNIQUE_ID', 'ALGORITHM'], 
                            columns='METRIC', 
                            values='VALUE', 
                            aggfunc='first').reset_index()
st.header("Per Series Metrics")
st.dataframe(metricsbyseries)


#Plot
chart_with_properties = alt.Chart(train).mark_line().encode(
    x='DS:T',
    y='Y:Q',
    tooltip=['DS', 'Y']
).properties(
    width=700,
    height=400,
    title='Full Time Series'
)
st.altair_chart(chart_with_properties, use_container_width=True)

st.write(selected_ag)
#Actual versus Predicted Plot
actuals.columns = ['UNIQUE_ID','VALUE','DS']
predicted.columns = ['DS','VALUE']
actuals['type'] = 'Actual'
predicted['type'] = 'Forecast'

actuals['DS'] = pd.to_datetime(actuals['DS'], errors='coerce')
# Find the latest date in your DataFrame
latest_date = actuals['DS'].max()
# Calculate the date 90 days before the latest date
ninety_days_ago = latest_date - pd.Timedelta(days=60)
# Filter the DataFrame to include only the last 90 days
last_90_days_df = actuals[actuals['DS'] >= ninety_days_ago]

combined_df = pd.concat([last_90_days_df, predicted])

chart = alt.Chart(combined_df).mark_line().encode(
    x='DS:T',
    y='VALUE:Q',
    color='type:N',
    tooltip=['DS', 'VALUE', 'type']
).properties(
    width=700,
    height=400,
    title='Actual vs Forecast - last 90 days'
)
st.altair_chart(chart, use_container_width=True)



st.header("Overall Metrics Across All Series")

metrics_sdf = session.read.table('TS_M5_metrics')
overall_performance = metrics_sdf.toPandas()
overall_performancedf = overall_performance.pivot_table(index=['ALGORITHM'], 
                            columns='METRIC', 
                            values='VALUE', 
                            aggfunc='first').reset_index()
st.dataframe(overall_performancedf)


# Assuming overall_performance is the DataFrame with 'Algorithm', 'Metric', and 'Value' columns

# Plot a separate bar chart for each metric
for metric in overall_performance['METRIC'].unique():
    # Filter the DataFrame for the current metric
    metric_data = overall_performance[overall_performance['METRIC'] == metric]
    
    # Create a bar chart in Altair
    chart1 = alt.Chart(metric_data).mark_bar().encode(
        x='ALGORITHM:N',
        y='VALUE:Q',
        color='ALGORITHM:N',
        tooltip=['ALGORITHM', 'VALUE']
    ).properties(
        title=f'Overall Performance for {metric}',
        width=600,
        height=400
    )
    
    # Display the plot in Streamlit
    st.altair_chart(chart1, use_container_width=True)
